In [1]:
from __future__ import division

import uproot
import matplotlib
import matplotlib.pyplot as plt
import numpy as np

plt.style.use('seaborn-deep')

In [2]:
data_ntuple = uproot.open('data.root')['SvtHits']
mc_ntuple = uproot.open('mc.root')['SvtHits']
mc_beam_ntuple = uproot.open('mc_beam.root')['SvtHits']

In [3]:
def draw_histogram1D(ax, variable, x_min, x_max, bins, **kargs):
    
    label = ''
    if 'label' in kargs: 
        label = kargs['label']
        
    norm = False
    if 'normalize' in kargs: 
        norm = kargs['normalize']
        
    ls='-'
    if 'linestyle' in kargs: 
        ls = kargs['linestyle']
        
    
    matplotlib.rcParams.update({'font.size': 22})
    
    x_bins = np.linspace(x_min, x_max, bins)
    if 'color' in kargs: 
        ax.hist(variable, bins=x_bins, histtype='step', linewidth=2, label=label, normed=norm, 
                ls=ls, color=kargs['color'])
    else: 
        ax.hist(variable, bins=x_bins, histtype='step', linewidth=2, label=label, normed=norm, ls=ls)
    
    if 'x_label' in kargs: 
        ax.set_xlabel(kargs['x_label'], fontsize=30)
    
    ax.minorticks_on()
    ax.tick_params(which='major', length=10, width=2, direction='inout', labelsize=30)
    ax.tick_params(which='minor', length=5, width=2, direction='in', labelsize=30)
    
    ax.set_title('$\it{HPS}$ $\it{Simulation}$ $\it{Internal}$', x=.82)
    
def error(bins,edges):
    
    # Just estimate the error as the sqrt of the count
    err = [np.sqrt(x) for x in bins]
    errmin = []
    errmax = []
    for x,err in zip(bins,err):
        errmin.append(x-err/2)
        errmax.append(x+err/2)

    return errmin, errmax

def ratio_error(bins1, bins2):
    
    err1 = np.array([np.true_divide(np.sqrt(x), x) for x in bins1])
    err2 = np.array([np.true_divide(np.sqrt(x), x) for x in bins2])
    
    err = np.sqrt(err1*err1 + err2*err2)
    
    bins = get_ratio(bins1, bins2)
    

    errmin = []
    errmax = []
    for x,err in zip(bins,err):
        errmin.append(x-err/2)
        errmax.append(x+err/2)

    return errmin, errmax

def get_ratio(bin1,bin2):
    # Sanity check
    if len(bin1) != len(bin2):
        print("Cannot make ratio!")
    
    bins = []
    for b1,b2 in zip(bin1,bin2):
        if b1==0 and b2==0:
            bins.append(1.)
        elif b2==0:
            bins.append(0.)
        else:
            bins.append(float(b1)/float(b2))
    
    # The ratio can of course be expanded with eg. error
    return bins

def ratio_error(bins1, bins2):
    
    err1 = np.array([np.true_divide(np.sqrt(x), x) for x in bins1])
    err2 = np.array([np.true_divide(np.sqrt(x), x) for x in bins2])
    
    err = np.sqrt(err1*err1 + err2*err2)

    return err

    
def data_mc_overlay(data, mc, x_min, x_max, bins, **kargs): 
    
        label = ''
        if 'label' in kargs: 
            label = kargs['label']
    
        x_bins = np.linspace(x_min, x_max, bins)
        dbins, dedges = np.histogram(data, bins=x_bins)
        mcbins, mcedges = np.histogram(mc, bins=x_bins)
        
        # Find the integrals of each of the histograms
        dint  = dbins.sum()
        mcint = mcbins.sum()
        
        derrmin, derrmax = error(dbins, dedges)
        mcerrmin, mcerrmax = error(mcbins, mcedges)
        rerr = ratio_error(dbins, mcbins)
        
        derrmin = np.true_divide(derrmin, dint)
        derrmax = np.true_divide(derrmax, dint)
        mcerrmin = np.true_divide(mcerrmin, mcint)
        mcerrmax = np.true_divide(mcerrmax, mcint)
        
        # Normalize each bin to the integral
        dbins  = np.true_divide(dbins,  dint)
        mcbins = np.true_divide(mcbins, mcint)
        
        dedges = dedges[:len(dedges)-1]
        mcedges = mcedges[:len(mcedges)-1]
        
        fig = plt.figure(figsize=(16, 12), facecolor='white')
        grid = plt.GridSpec(5, 1, hspace=0.1, wspace=0.2)
        
        # Plot as colored o's, connected
        ax = fig.add_subplot(grid[:4, 0])
        ax_r = fig.add_subplot(grid[4, 0], sharex=ax)
        plt.setp(ax.get_xticklabels(), visible=False)
        
        ax.plot(dedges, dbins, 'o', color='#4C72B0', lw=2, label='Data')
        ax.step(mcedges, mcbins, where='mid', color='#55A868', lw=2, label='MC')

        leg = ax.legend()
        
        # Plot the error bars
        ax.vlines(dedges, derrmin, derrmax, color='#4C72B0')
        ax.vlines(mcedges, mcerrmin, mcerrmax, color='#55A868')
        
        ax.set_title('$\it{HPS}$ $\it{Internal}$', x=.82)

        # Plot the ratio

        rat = get_ratio(dbins, mcbins)
        
        rerrmin = []
        rerrmax = []
        for x,err in zip(rat, rerr):
            rerrmin.append(x-err/2)
            rerrmax.append(x+err/2)
        
        ax_r.plot(dedges,rat, 'o', color='black')
        ax_r.vlines(dedges, rerrmin, rerrmax, color='black')
        ax_r.set_ylim(0,2)
        ax_r.set_ylabel('Data/MC')
        
        if 'x_label' in kargs: 
            ax_r.set_xlabel(kargs['x_label'], fontsize=30)
    
        #ax.set_xlabel("Variable name")
    

In [4]:
data_hit_arr  = {}
data_hit_farr = {}
for key in data_ntuple.keys():
    data_hit_arr[key]  = data_ntuple.array(key)
    data_hit_farr[key] = data_hit_arr[key].flatten()
    
mc_hit_arr  = {}
mc_hit_farr = {}
for key in mc_ntuple.keys(): 
    mc_hit_arr[key]  = mc_ntuple.array(key)
    mc_hit_farr[key] = mc_hit_arr[key].flatten() 
    
mc_beam_arr  = {}
mc_beam_farr = {}
for key in mc_beam_ntuple.keys(): 
    mc_beam_arr[key]  = mc_beam_ntuple.array(key)
    mc_beam_farr[key] = mc_beam_arr[key].flatten() 

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(16, 10), facecolor='white')
draw_histogram1D(ax, mc_beam_farr['sim_hit_count'],   0, 270, 271, x_label="Hit Multiplicity, All Layers", label="Sim Hits")
draw_histogram1D(ax, mc_beam_farr['raw_hit_count'],   0, 270, 271, label="Raw Hits")
draw_histogram1D(ax, mc_beam_farr['strip_cluster_count'], 0, 270, 271, label="1D Strip Hits")

ax.legend(loc=1, fontsize=30)

In [ ]:
#fig, ax = plt.subplots(2, 14, sharex='col', sharey='row')
#for i in range(14): 
#    draw_histogram(ax[0, i], mc_hit_farr['raw_hit_count'][]

In [ ]:
data_mc_overlay(data_hit_farr['raw_hit_count'],  mc_beam_farr['raw_hit_count'], 
                0, 400, 201, x_label='Raw Hit Multiplicity, All Layers' )

In [ ]:
data_mc_overlay(data_hit_farr['strip_cluster_count'],  mc_beam_farr['strip_cluster_count'], 
                0, 200, 101, x_label='1D Strip Hit Multiplicity, All Layers' )

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(16, 10), facecolor='white', sharex='col', sharey='row')
fig.subplots_adjust(hspace=0.05, wspace=0.05)


cut = (mc_beam_farr['raw_hit_is_top'] == 1) & (mc_beam_farr['raw_hit_is_ele_side'] == 1)
draw_histogram1D(ax, mc_beam_farr['raw_hit_layer'][cut] - 1, 0, 14, 15, 
                 label="Top", color='#55A868', x_label='Layer')
cut = (mc_beam_farr['raw_hit_is_top'] == 1) & (mc_beam_farr['raw_hit_is_pos_side'] == 1)
draw_histogram1D(ax, mc_beam_farr['raw_hit_layer'][cut] - 1, 0, 14, 15, linestyle='--', color='#55A868')

cut = (mc_beam_farr['raw_hit_is_bot'] == 1) & (mc_beam_farr['raw_hit_is_ele_side'] == 1)
draw_histogram1D(ax, mc_beam_farr['raw_hit_layer'][cut] - 1, 0, 14, 15, 
                 label="Bottom", color='#4C72B0')
cut = (mc_beam_farr['raw_hit_is_bot'] == 1) & (mc_beam_farr['raw_hit_is_pos_side'] == 1)
draw_histogram1D(ax, mc_beam_farr['raw_hit_layer'][cut] - 1, 0, 14, 15, linestyle='--', color='#4C72B0')

ax.legend(loc=1, fontsize=30)

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(16, 10), facecolor='white', sharex='col', sharey='row')
fig.subplots_adjust(hspace=0.05, wspace=0.05)


cut = (mc_beam_farr['strip_cluster_is_top'] == 1) & (mc_beam_farr['strip_cluster_is_ele_side'] == 1)
draw_histogram1D(ax, mc_beam_farr['strip_cluster_layer'][cut] - 1, 0, 14, 15, 
                 label="Top", color='#55A868', x_label='Layer')
cut = (mc_beam_farr['strip_cluster_is_top'] == 1) & (mc_beam_farr['strip_cluster_is_pos_side'] == 1)
draw_histogram1D(ax, mc_beam_farr['strip_cluster_layer'][cut] - 1, 0, 14, 15, linestyle='--', color='#55A868')

cut = (mc_beam_farr['strip_cluster_is_bot'] == 1) & (mc_beam_farr['strip_cluster_is_ele_side'] == 1)
draw_histogram1D(ax, mc_beam_farr['strip_cluster_layer'][cut] - 1, 0, 14, 15, 
                 label="Bottom", color='#4C72B0')
cut = (mc_beam_farr['strip_cluster_is_bot'] == 1) & (mc_beam_farr['strip_cluster_is_pos_side'] == 1)
draw_histogram1D(ax, mc_beam_farr['strip_cluster_layer'][cut] - 1, 0, 14, 15, linestyle='--', color='#4C72B0')

ax.legend(loc=1, fontsize=30)

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(16, 10), facecolor='white', sharex='col', sharey='row')
fig.subplots_adjust(hspace=0.05, wspace=0.05)


cut = (mc_beam
       _farr['raw_hit_is_top'] == 1) & (mc_beam_farr['raw_hit_is_ele_side'] == 1)
draw_histogram1D(ax, mc_beam_farr['raw_hit_layer'][cut] - 1, 0, 14, 15, 
                 label="Top", color='#55A868', x_label='Layer')
cut = (mc_beam_farr['raw_hit_is_top'] == 1) & (mc_beam_farr['raw_hit_is_pos_side'] == 1)
draw_histogram1D(ax, mc_beam_farr['raw_hit_layer'][cut] - 1, 0, 14, 15, linestyle='--', color='#55A868')

cut = (mc_beam_farr['raw_hit_is_bot'] == 1) & (mc_beam_farr['raw_hit_is_ele_side'] == 1)
draw_histogram1D(ax, mc_beam_farr['raw_hit_layer'][cut] - 1, 0, 14, 15, 
                 label="Bottom", color='#4C72B0')
cut = (mc_beam_farr['raw_hit_is_bot'] == 1) & (mc_beam_farr['raw_hit_is_pos_side'] == 1)
draw_histogram1D(ax, mc_beam_farr['raw_hit_layer'][cut] - 1, 0, 14, 15, linestyle='--', color='#4C72B0
                 ax.legend(loc=1, fontsize=30)                              

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(16, 10), facecolor='white')
cut = (mc_hit_farr['raw_hit_layer'] == 1) & (mc_hit_farr['raw_hit_is_top'] == 1)
draw_histogram1D(ax, mc_hit_farr['raw_hit_strip'][cut], 0, 639, 640,
                 x_label='Physical Channel', label='Layer 0 - Top - Axial', normalize=True)

cut = (mc_hit_farr['raw_hit_layer'] == 5) & (mc_hit_farr['raw_hit_is_top'] == 1)
draw_histogram1D(ax, mc_hit_farr['raw_hit_strip'][cut], 0, 639, 640,
                 label='Layer 2 - Top - Axial', normalize=True)


ax.set_yscale("log")
ax.legend(loc=9, fontsize=30)

In [ ]:
dcut = (data_hit_farr['raw_hit_layer'] == 1) & (data_hit_farr['raw_hit_is_top'] == 1)
mccut = (mc_beam_farr['raw_hit_layer'] == 1) & (mc_beam_farr['raw_hit_is_top'] == 1)
data_mc_overlay(data_hit_farr['raw_hit_strip'][dcut],  mc_beam_farr['raw_hit_strip'][mccut], 
                0, 639, 640, x_label='Physical Channel' )

In [ ]:
dcut = (data_hit_farr['raw_hit_layer'] == 5) & (data_hit_farr['raw_hit_is_top'] == 1)
mccut = (mc_beam_farr['raw_hit_layer'] == 5) & (mc_beam_farr['raw_hit_is_top'] == 1)
data_mc_overlay(data_hit_farr['raw_hit_strip'][dcut],  mc_beam_farr['raw_hit_strip'][mccut], 
                0, 639, 640, x_label='Physical Channel' )

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(16, 10), facecolor='white')
cut = data_hit_farr['raw_hit_layer'] <=4
anti_cut = data_hit_farr['raw_hit_layer'] > 4
draw_histogram1D(ax, data_hit_farr['raw_hit_strip'][anti_cut], 0, 639, 640, x_label='Physical Channel', label='Layer > 2')
draw_histogram1D(ax, data_hit_farr['raw_hit_strip'][cut], 0, 639, 640, label='Layer 0/1')

ax.legend(loc=2, fontsize=30)

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(16, 10), facecolor='white')

x_bins, y_bins = np.linspace(0, 5, 6), np.linspace(4000, 8000, 2001)
sample1 = np.zeros(len(mc_hit_farr['raw_hit_layer']))
sample2 = np.ones(len(mc_hit_farr['raw_hit_layer']))
sample3 = np.ones(len(mc_hit_farr['raw_hit_layer']))*2
sample4 = np.ones(len(mc_hit_farr['raw_hit_layer']))*3
sample5 = np.ones(len(mc_hit_farr['raw_hit_layer']))*4
sample6 = np.ones(len(mc_hit_farr['raw_hit_layer']))*5
cut = ((mc_hit_farr['raw_hit_layer'] == 1) & (mc_hit_farr['raw_hit_is_top'] == 1))
ax.hist2d(sample1[cut], mc_hit_farr['raw_hit_adc0'][cut], bins=[x_bins, y_bins], cmap='jet', norm=matplotlib.colors.LogNorm())
ax.hist2d(sample2[cut], mc_hit_farr['raw_hit_adc1'][cut], bins=[x_bins, y_bins], cmap='jet', norm=matplotlib.colors.LogNorm())
ax.hist2d(sample3[cut], mc_hit_farr['raw_hit_adc2'][cut], bins=[x_bins, y_bins], cmap='jet', norm=matplotlib.colors.LogNorm());
ax.hist2d(sample4[cut], mc_hit_farr['raw_hit_adc3'][cut], bins=[x_bins, y_bins], cmap='jet', norm=matplotlib.colors.LogNorm());
ax.hist2d(sample5[cut], mc_hit_farr['raw_hit_adc4'][cut], bins=[x_bins, y_bins], cmap='jet', norm=matplotlib.colors.LogNorm());
ax.hist2d(sample6[cut], mc_hit_farr['raw_hit_adc5'][cut], bins=[x_bins, y_bins], cmap='jet', norm=matplotlib.colors.LogNorm());
ax.set_xlabel('Sample Number')
ax.set_ylabel('Amplitude (ADC Counts)')
ax.set_title('$\it{HPS}$ $\it{Simulation}$ $\it{Internal}$', x=.82)

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(16, 10), facecolor='white')

x_bins, y_bins = np.linspace(0, 5, 6), np.linspace(4000, 8000, 2001)
sample1 = np.zeros(len(mc_hit_farr['raw_hit_layer']))
sample2 = np.ones(len(mc_hit_farr['raw_hit_layer']))
sample3 = np.ones(len(mc_hit_farr['raw_hit_layer']))*2
sample4 = np.ones(len(mc_hit_farr['raw_hit_layer']))*3
sample5 = np.ones(len(mc_hit_farr['raw_hit_layer']))*4
sample6 = np.ones(len(mc_hit_farr['raw_hit_layer']))*5
cut = ((mc_hit_farr['raw_hit_layer'] == 5) & (mc_hit_farr['raw_hit_is_top'] == 1))
ax.hist2d(sample1[cut], mc_hit_farr['raw_hit_adc0'][cut], bins=[x_bins, y_bins], cmap='jet', norm=matplotlib.colors.LogNorm())
ax.hist2d(sample2[cut], mc_hit_farr['raw_hit_adc1'][cut], bins=[x_bins, y_bins], cmap='jet', norm=matplotlib.colors.LogNorm())
ax.hist2d(sample3[cut], mc_hit_farr['raw_hit_adc2'][cut], bins=[x_bins, y_bins], cmap='jet', norm=matplotlib.colors.LogNorm());
ax.hist2d(sample4[cut], mc_hit_farr['raw_hit_adc3'][cut], bins=[x_bins, y_bins], cmap='jet', norm=matplotlib.colors.LogNorm());
ax.hist2d(sample5[cut], mc_hit_farr['raw_hit_adc4'][cut], bins=[x_bins, y_bins], cmap='jet', norm=matplotlib.colors.LogNorm());
ax.hist2d(sample6[cut], mc_hit_farr['raw_hit_adc5'][cut], bins=[x_bins, y_bins], cmap='jet', norm=matplotlib.colors.LogNorm());
ax.set_xlabel('Sample Number')
ax.set_ylabel('Amplitude (ADC Counts)')
ax.set_title('$\it{HPS}$ $\it{Simulation}$ $\it{Internal}$', x=.82)

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(16, 10), facecolor='white')

x_bins, y_bins = np.linspace(-25, 25, 100), np.linspace(0, 639, 640)
cut = (mc_hit_farr['sim_hit_layer'] == 1) & (mc_hit_farr['sim_hit_is_top'] == 1)
ax.hist2d(mc_hit_farr['sim_hit_x'][cut], mc_hit_farr['sim_hit_raw_strip'][cut], bins=[x_bins, y_bins], cmap='jet', norm=matplotlib.colors.LogNorm());

ax.set_title('$\it{HPS}$ $\it{Simulation}$ $\it{Internal}$', x=.82)
ax.set_xlabel('SimTrackerHit x (mm)', fontsize=30)
ax.set_ylabel('Physical Channel', fontsize=30)

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(16, 10), facecolor='white')

x_bins, y_bins = np.linspace(-25, 25, 100), np.linspace(0, 639, 640)
cut = ((mc_hit_farr['sim_hit_layer'] == 1) & (mc_hit_farr['sim_hit_y'] < 0))
ax.hist2d(mc_hit_farr['sim_hit_x'][cut], mc_hit_farr['sim_hit_raw_strip'][cut], bins=[x_bins, y_bins], cmap='jet', norm=matplotlib.colors.LogNorm());

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(16, 10), facecolor='white')

x_bins, y_bins = np.linspace(-25, 25, 100), np.linspace(0, 639, 640)
cut = (mc_hit_farr['sim_hit_layer'] == 1) & (mc_hit_farr['sim_hit_is_top'] == 1)
ax.hist2d(mc_hit_farr['sim_hit_y'][cut], mc_hit_farr['sim_hit_raw_strip'][cut], bins=[x_bins, y_bins], cmap='jet', norm=matplotlib.colors.LogNorm());

cut = (mc_hit_farr['sim_hit_layer'] == 2) & (mc_hit_farr['sim_hit_is_bot'] == 1)
ax.hist2d(mc_hit_farr['sim_hit_y'][cut], mc_hit_farr['sim_hit_raw_strip'][cut], bins=[x_bins, y_bins], cmap='jet', norm=matplotlib.colors.LogNorm());

ax.set_title('$\it{HPS}$ $\it{Simulation}$ $\it{Internal}$', x=.82)
ax.set_xlabel('SimTrackerHit y (mm)', fontsize=30)
ax.set_ylabel('Physical Channel', fontsize=30)

In [ ]:
matplotlib.rcParams.update({'font.size': 22})

fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(16, 10), facecolor='white')

x_bins, y_bins = np.linspace(-100, 100, 100), np.linspace(-100, 100, 100)
ax.hist2d(mc_beam_farr['sim_hit_x'], mc_beam_farr['sim_hit_y'], bins=[x_bins, y_bins], cmap='jet', norm=matplotlib.colors.LogNorm());

In [ ]:
matplotlib.rcParams.update({'font.size': 22})

fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(16, 10), facecolor='white')

x_bins, y_bins = np.linspace(-25, 25, 100), np.linspace(-25, 25, 100)

cut = hit_farr['sim_hit_layer'] == 1

ax.hist2d(hit_farr['sim_hit_x'][cut], 
          hit_farr['sim_hit_y'][cut], 
          bins=[x_bins, y_bins], cmap='jet', norm=matplotlib.colors.LogNorm());

In [ ]:
matplotlib.rcParams.update({'font.size': 22})

fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(16, 10), facecolor='white')

x_bins, y_bins = np.linspace(-100, 100, 100), np.linspace(-100, 100, 100)
ax.hist2d(hit_farr['strip_hit_y'], hit_farr['strip_hit_x'], bins=[x_bins, y_bins], cmap='jet', norm=matplotlib.colors.LogNorm());

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(16, 10), facecolor='white')
cut = (mc_hit_farr['strip_cluster_is_bot'] == 1)
draw_histogram1D(ax, mc_hit_farr['strip_cluster_z'][cut], 0, 1000, 500, x_label='Strip Hit z (mm)', label='Bottom')
cut = (mc_hit_farr['strip_cluster_is_top'] == 1)
draw_histogram1D(ax, mc_hit_farr['strip_cluster_z'][cut], 0, 1000, 500, label='Top')

ax.legend(loc=1, fontsize=30)

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(16, 10), facecolor='white')
cut = (data_hit_farr['strip_hit_y'] < 0)
draw_histogram1D(ax, data_hit_farr['strip_hit_z'][cut], 0, 1000, 500, x_label='Strip Hit z (mm)', label='Bottom')
cut = (data_hit_farr['strip_hit_y'] > 0)
draw_histogram1D(ax, data_hit_farr['strip_hit_z'][cut], 0, 1000, 500, label='Top')

ax.legend(loc=1, fontsize=30)

In [ ]:
matplotlib.rcParams.update({'font.size': 22})

fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(16, 10), facecolor='white')

x_bins, y_bins = np.linspace(-30, 20, 100), np.linspace(-40, 20, 100)
cut = ((hit_farr['strip_hit_y'] < 0) & (hit_farr['strip_hit_z'] < 55))
ax.hist2d(hit_farr['strip_hit_x'][cut], hit_farr['strip_hit_y'][cut], bins=[x_bins, y_bins], cmap='jet', norm=matplotlib.colors.LogNorm());

In [ ]:
dcut = (data_hit_farr['strip_cluster_is_top'] == 1) & (data_hit_farr['strip_cluster_layer'] == 1)
mccut = (mc_beam_farr['strip_cluster_is_top'] == 1) & (mc_beam_farr['strip_cluster_layer'] == 1)

data_mc_overlay(data_hit_farr['strip_cluster_size'][dcut],  mc_beam_farr['strip_cluster_size'][mccut], 
                0, 10, 11, x_label='Strip Cluster Size' )


In [ ]:
dcut = (data_hit_farr['strip_cluster_is_top'] == 1) & (data_hit_farr['strip_cluster_layer'] == 5)
mccut = (mc_beam_farr['strip_cluster_is_top'] == 1) & (mc_beam_farr['strip_cluster_layer'] == 5)

data_mc_overlay(data_hit_farr['strip_cluster_size'][dcut],  mc_beam_farr['strip_cluster_size'][mccut], 
                0, 10, 11, x_label='Strip Cluster Size' )


In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(16, 10), facecolor='white')
cut = (mc_beam_farr['sim_hit_layer'] == 1) & (mc_beam_farr['sim_hit_is_top'] == 1) & (mc_beam_farr['sim_hit_raw_strip'] < 256)
draw_histogram1D(ax, mc_beam_farr['sim_hit_strip_res_x'][cut], -25, 25, 101, 
                 x_label='Sim Hit x - Strip Hit x (mm), Layer 0 - Top - Axial', label='Channel < 256')

cut = (mc_beam_farr['sim_hit_layer'] == 1) & (mc_beam_farr['sim_hit_is_top'] == 1) & (mc_beam_farr['sim_hit_raw_strip'] >= 256)
draw_histogram1D(ax, mc_beam_farr['sim_hit_strip_res_x'][cut], -25, 25, 101, label='Channel >= 256')

ax.legend(loc=2, fontsize=30)

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(16, 10), facecolor='white')
cut = (mc_beam_farr['sim_hit_layer'] == 1) & (mc_beam_farr['sim_hit_is_top'] == 1) & (mc_beam_farr['sim_hit_raw_strip'] < 256) & (mc_beam_farr['sim_hit_strip_size'] > 1)
cut = cut & (mc_beam_farr['sim_hit_strip_res_x'] != -9999)
draw_histogram1D(ax, mc_beam_farr['sim_hit_strip_res_x'][cut]/mc_beam_farr['sim_hit_strip_res_xerr'][cut], -2, 2, 101, 
                 x_label='Pull, Layer 0 - Top - Axial', label='Channel < 256')

cut = (mc_beam_farr['sim_hit_layer'] == 1) & (mc_beam_farr['sim_hit_is_top'] == 1) & (mc_beam_farr['sim_hit_raw_strip'] >= 256) & (mc_beam_farr['sim_hit_strip_size'] > 1)
cut = cut & (mc_beam_farr['sim_hit_strip_res_x'] != -9999)
draw_histogram1D(ax, mc_beam_farr['sim_hit_strip_res_x'][cut]/mc_beam_farr['sim_hit_strip_res_xerr'][cut], -2, 2, 101, label='Channel >= 256')

ax.legend(loc=2, fontsize=30)

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(16, 10), facecolor='white')
cut = (mc_beam_farr['sim_hit_layer'] == 5) & (mc_beam_farr['sim_hit_is_top'] == 1) & (mc_beam_farr['sim_hit_strip_size'] > 1)
draw_histogram1D(ax, mc_beam_farr['sim_hit_strip_res_x'][cut], -25, 25, 101, 
                 x_label='Sim Hit x - Strip Hit x (mm), Layer 2 - Top - Axial')

#ax.legend(loc=2, fontsize=30)

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(16, 10), facecolor='white')
cut = (mc_beam_farr['sim_hit_layer'] == 5) & (mc_beam_farr['sim_hit_is_top'] == 1) & (mc_beam_farr['sim_hit_strip_size'] > 1)
draw_histogram1D(ax, mc_beam_farr['sim_hit_strip_res_x'][cut], -10, 10, 101, 
                 x_label='Pull, Layer 2 - Top - Axial')

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(16, 10), facecolor='white')
cut = (mc_beam_farr['sim_hit_layer'] == 1) & (mc_beam_farr['sim_hit_is_top'] == 1) & (mc_beam_farr['sim_hit_raw_strip'] < 256)
draw_histogram1D(ax, mc_beam_farr['sim_hit_strip_res_y'][cut], -0.15, 0.15, 101, 
                 x_label='Sim Hit y - Strip Hit y (mm), Layer 0 - Top - Axial', label='Channel < 256')
#cut = cut & (mc_beam_farr['sim_cluster_size'] > 1)

cut = (mc_beam_farr['sim_hit_layer'] == 1) & (mc_beam_farr['sim_hit_is_top'] == 1) & (mc_beam_farr['sim_hit_raw_strip'] >= 256)
draw_histogram1D(ax, mc_beam_farr['sim_hit_strip_res_y'][cut], -0.15, 0.15, 101, label='Channel >= 256')

ax.legend(loc=2, fontsize=30)

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(16, 10), facecolor='white')
cut = (mc_beam_farr['sim_hit_layer'] == 5) & (mc_beam_farr['sim_hit_is_top'] == 1) 
draw_histogram1D(ax, mc_beam_farr['sim_hit_strip_res_y'][cut], -.15, .15, 101, 
                 x_label='Sim Hit x - Strip Hit x (mm), Layer 2 - Top - Axial')

#ax.legend(loc=2, fontsize=30)

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(16, 10), facecolor='white')
cut = (mc_beam_farr['sim_hit_layer'] == 1) & (mc_beam_farr['sim_hit_is_top'] == 1) & (mc_beam_farr['sim_hit_raw_strip'] < 256) 
cut = cut & (mc_beam_farr['sim_hit_strip_res_y'] != -9999)
draw_histogram1D(ax, mc_beam_farr['sim_hit_strip_res_y'][cut]/mc_beam_farr['sim_hit_strip_res_yerr'][cut], -2, 2, 101, 
                 x_label='Pull, Layer 0 - Top - Axial', label='Channel < 256')

cut = (mc_beam_farr['sim_hit_layer'] == 1) & (mc_beam_farr['sim_hit_is_top'] == 1) & (mc_beam_farr['sim_hit_raw_strip'] >= 256)
cut = cut & (mc_beam_farr['sim_hit_strip_res_y'] != -9999)
draw_histogram1D(ax, mc_beam_farr['sim_hit_strip_res_y'][cut]/mc_beam_farr['sim_hit_strip_res_yerr'][cut], -2, 2, 101, label='Channel >= 256')

ax.legend(loc=2, fontsize=30)

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(16, 10), facecolor='white')
cut = (mc_beam_farr['sim_hit_layer'] == 5) & (mc_beam_farr['sim_hit_is_top'] == 1)
cut = cut & (mc_beam_farr['sim_hit_strip_res_y'] != -9999)
draw_histogram1D(ax, mc_beam_farr['sim_hit_strip_res_y'][cut]/mc_beam_farr['sim_hit_strip_res_yerr'][cut], -2, 2, 101, 
                 x_label='Pull, Layer 2 - Top - Axial', label='Channel < 256')